<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/pi05-test/third_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Third Test

This time we have a more robust auto regressive implemenation, because now I understand the attention masking. Both full and causal.

In [4]:
from huggingface_hub import login
login()

In [1]:
# Clone

Cloning into 'XHUMAN'...
remote: Enumerating objects: 2098, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 2098 (delta 196), reused 195 (delta 110), pack-reused 1794 (from 2)
Receiving objects: 100% (2098/2098), 7.82 MiB | 11.40 MiB/s, done.
Resolving deltas: 100% (1339/1339), done.


In [2]:
%cd XHUMAN

/content/XHUMAN


We have several problem trying to use the transformers library, this requires a more careful study

In [3]:
!uv pip install -e .[pi]

Using Python 3.12.12 environment at: /usr
Resolved 215 packages in 27.24s
Prepared 46 packages in 21.15s
Uninstalled 15 packages in 967ms
Installed 46 packages in 219ms
 + async-lru==2.0.5
 + av==15.1.0
 + comm==0.2.3
 + deepdiff==8.6.1
 - diffusers==0.36.0
 + diffusers==0.35.2
 + draccus==0.10.0
 + evdev==1.9.2
 + faker==40.1.2
 + feetech-servo-sdk==1.0.0
 - huggingface-hub==0.36.0
 + huggingface-hub==0.35.3
 + inquirerpy==0.3.4
 - ipykernel==6.17.1
 + ipykernel==7.1.0
 + jedi==0.19.2
 + json5==0.13.0
 + jsonlines==4.0.0
 + jupyter==1.1.1
 - jupyter-client==7.4.9
 + jupyter-client==8.8.0
 + jupyter-lsp==2.3.0
 + jupyterlab==4.5.2
 + jupyterlab-server==2.28.0
 + lerobot==0.4.3 (from git+https://github.com/huggingface/lerobot.git@46e19ae579f80ce66211afafd1c3c649c569131f)
 + mergedeep==1.3.4
 + mypy-extensions==1.1.0
 - notebook==6.5.7
 + notebook==7.5.2
 - nvidia-cudnn-cu12==9.10.2.21
 + nvidia-cudnn-cu12==9.5.1.17
 - nvidia-cusparselt-cu12==0.7.1
 + nvidia-cusparselt-cu12==0.6.3
 - nvi

# Convert the image into something workable

In [7]:
from transformers import AutoTokenizer
# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("google/paligemma-3b-pt-224",
                                          use_fast=False)

Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [19]:
import os
import cv2
import torch

img_share_path = '/content'
img_name_list = ['top.png']
img_list = []

if img_share_path and os.path.exists(img_share_path):
    # Load images from path
    for img_name in img_name_list:
        img_path = os.path.join(img_share_path, img_name)
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            if img is not None:
                # Convert BGR to RGB
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_list.append(img)

In [25]:
from lerobot.utils.constants import OBS_LANGUAGE_TOKENS, OBS_LANGUAGE_ATTENTION_MASK

batch = {}
if len(img_list) > 0:
    # Prepare image tensor: [H, W, C] -> [1, H, W, C]
    # Normalize to [0, 1] as expected by _preprocess_images (which then scales to [-1, 1])
    img_tensor = torch.from_numpy(img_list[0]).float() / 255.0
    img_tensor = img_tensor.unsqueeze(0) # Add batch dimension -> [1, H, W, C]

    batch["observation.images.top"] = img_tensor

# Process Language Prompt
# Note: PI05 automatically prepends image embeddings, so we DO NOT need an <image> token in the text.
# However, we ensure the prompt ends with a newline for better model performance.
prompt = "Put the fruits in the basket\nSub task: "
if not prompt.endswith("\n"):
    prompt += "\n"

print(f"Tokenizing prompt: '{prompt.strip()}'")
tokenized = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
)

# Add to batch (converting mask to bool for compatibility)
batch[OBS_LANGUAGE_TOKENS] = tokenized["input_ids"]
batch[OBS_LANGUAGE_ATTENTION_MASK] = tokenized["attention_mask"].bool()

Tokenizing prompt: 'Put the fruits in the basket
Sub task:'


In [10]:
from xhuman.policies.pi05.modeling_pi_05 import PI05Policy

model_id = "NONHUMAN-RESEARCH/pi05_ki_training_fruits"

policy = PI05Policy.from_pretrained(model_id)

config.json: 0.00B [00:00, ?B/s]

processor_config.json:   0%|          | 0.00/253 [00:00<?, ?B/s]

processing_action_tokenizer.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/physical-intelligence/fast:
- processing_action_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/322 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

Loading model from: NONHUMAN-RESEARCH/pi05_ki_training_fruits


model.safetensors:   0%|          | 0.00/7.47G [00:00<?, ?B/s]

✓ Loaded state dict from model.safetensors
	Missing key(s) in state_dict: "model.paligemma_with_expert.paligemma.model.language_model.embed_tokens.weight". 


In [12]:
device = "cuda"
policy.model.eval()
policy.model.to(device)

AttributeError: 'GemmaRMSNorm' object has no attribute 'weight'

In [32]:
batch['observation.images.top'].shape

torch.Size([1, 768, 1366, 3])

In [38]:
processed_images, img_masks = policy._preprocess_images(batch)

In [39]:
processed_images[0].shape

torch.Size([1, 224, 224, 3])

In [40]:
tokens = batch[OBS_LANGUAGE_TOKENS].to(device)
masks = batch[OBS_LANGUAGE_ATTENTION_MASK].to(device)

In [41]:
max_decoding_steps = 100
temperature = 0.1
PALIGEMMA_EOS_TOKEN = 1

In [48]:
tokens

tensor([[     2,  15360,    573,  16803,    575,    573,  12220,    108,   3351,
           6911, 235292, 235248,    108]], device='cuda:0')

In [50]:
masks

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True]], device='cuda:0')

In [43]:
with torch.no_grad():
    # FIX: Ensure images are in (B, C, H, W) format
    # The error "expected input... to have 3 channels, but got 224" occurs because
    # the input was (B, H, W, C) but the model expects (B, C, H, W).
    fixed_processed_images = []
    for img in processed_images:
        # Check if shape is (B, H, W, C=3) instead of (B, C=3, H, W)
        if img.ndim == 4 and img.shape[-1] == 3 and img.shape[1] != 3:
            img = img.permute(0, 3, 1, 2)
        fixed_processed_images.append(img)

    subtask_tokenized = policy.model.sample_subtask(
        images=fixed_processed_images, # Use the fixed images
        img_masks=img_masks,
        tokens=tokens,
        masks=masks.bool(),
        max_decoding_steps=max_decoding_steps,
        eos_token_id=PALIGEMMA_EOS_TOKEN,
        temperature=temperature
    )

In [44]:
subtask_tokenized

tensor([[3351,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]], device='cuda:0')

In [47]:
sub = tokenizer.decode(subtask_tokenized[0], skip_special_tokens=True)
print(sub)

Sub


In [ ]:
# _preprocess_images returns processed_images and img_masks
# processed_images: list of (B, C, H, W) tensors
# img_masks: list of (B,) boolean tensors

# Prepare language prompt

print(f"\nPrompt: {high_level_prompt}")

# Tokenize the prompt
# Add newline if not present
if not high_level_prompt.endswith('\n'):
    high_level_prompt += '\n'

tokenized = tokenizer(
    high_level_prompt,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
)

print(f"Tokenized: {tokenized}")

tokens = tokenized["input_ids"].to(device)  # (B, seq_len)
masks = tokenized["attention_mask"].to(device)  # (B, seq_len)

print(f"Tokens shape: {tokens.shape}")
print(f"Masks shape: {masks.shape}")


Prompt: Task: Put the grapes on the basket.
Subtask:
Tokenized: {'input_ids': tensor([[     2,   7071, 235292,  13298,    573,  50161,    611,    573,  12220,
         235265,    108,   3351,   8277, 235292,    108]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Tokens shape: torch.Size([1, 15])
Masks shape: torch.Size([1, 15])


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Visualize the processed images
# These images are normalized to [-1, 1] and might be resized

if 'processed_images' in locals():
    num_images = len(processed_images)
    plt.figure(figsize=(5 * num_images, 5))

    for i, img_tensor in enumerate(processed_images):
        # processed_images are typically (B, H, W, C) or (B, C, H, W)
        # We select the first image in the batch
        img = img_tensor[0].detach().cpu().numpy()

        # Check if channels are first (C, H, W) -> convert to (H, W, C) for plotting
        # Heuristic: if first dimension is 3, it's likely channels
        if img.ndim == 3 and img.shape[0] == 3:
             img = np.transpose(img, (1, 2, 0))

        # Un-normalize: [-1, 1] -> [0, 1]
        img = (img + 1.0) / 2.0
        img = np.clip(img, 0.0, 1.0)

        plt.subplot(1, num_images, i + 1)
        plt.imshow(img)
        plt.title(f"Image {i}\nShape: {img_tensor.shape}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()
else:
    print("processed_images variable not found.")

In [ ]:
with torch.no_grad():
    # FIX: Ensure images are in (B, C, H, W) format
    # The error "expected input... to have 3 channels, but got 224" occurs because
    # the input was (B, H, W, C) but the model expects (B, C, H, W).
    fixed_processed_images = []
    for img in processed_images:
        # Check if shape is (B, H, W, C=3) instead of (B, C=3, H, W)
        if img.ndim == 4 and img.shape[-1] == 3 and img.shape[1] != 3:
            img = img.permute(0, 3, 1, 2)
        fixed_processed_images.append(img)

    output_tokens, updated_past_key_values = policy.model.sample_low_level_task(
        images=fixed_processed_images,
        img_masks=img_masks,
        tokens=tokens,
        masks=masks.bool(), # FIX: Convert attention mask to boolean
        max_decoding_steps=max_decoding_steps,
        eos_token_id=PALIGEMMA_EOS_TOKEN,
        temperature=temperature
    )

    print(f"Output tokens shape: {output_tokens.shape}")
    print(f"Output tokens:\n{output_tokens}")
    print(f"Updated past key values: {updated_past_key_values}")

    # Decode the generated tokens
    print("\n" + "="*80)
    print("High Level Prompt\n",high_level_prompt)
    print("Decoded output:")
    print("="*80)

    for batch_idx in range(output_tokens.shape[0]):
        batch_tokens = output_tokens[batch_idx]
        # Remove padding (zeros) and decode
        non_zero_tokens = batch_tokens[batch_tokens != 0]
        if len(non_zero_tokens) > 0:
            decoded_text = tokenizer.decode(non_zero_tokens, skip_special_tokens=False)
            print(f"Batch {batch_idx}: {decoded_text}")
        else:
            print(f"Batch {batch_idx}: (empty)")

print("\n" + "="*80)
print("Test completed successfully!")
print("="*80)

Output tokens shape: torch.Size([1, 40])
Output tokens:
tensor([[18075,     1,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]],
       device='cuda:0')
Updated past key values: <transformers.cache_utils.DynamicCache object at 0x7e50cc953080>

High Level Prompt Task: Put the grapes on the basket.
Subtask:

Decoded output:
Batch 0: pick<eos>

Test completed successfully!


# Results

The model don't produce non sense words. But it just produce a word, maybe there is some bug with the EOS token. Or maybe the adaRMS thing. Before make a fine tunning we have to be sure about it.

### Fine-Tuning Example
To fine-tune the model, you need:
1. **Ground Truth Actions**: The actual movements the robot should make.
2. **Training Loop**: A standard PyTorch loop with an optimizer.

The following code demonstrates a single training step using dummy action data.

In [ ]:
import torch.optim as optim
from lerobot.utils.constants import ACTION

# 1. Setup Optimizer
# We fine-tune all parameters here, but you could also freeze specific layers
optimizer = optim.AdamW(policy.parameters(), lr=1e-5)

# 2. Prepare a Training Batch
# We start with the existing batch containing images and text
train_batch = batch.copy()

# We need to add Ground Truth Actions for the loss calculation
# Shape: (Batch_Size, Horizon, Action_Dim)
action_dim = policy.config.max_action_dim
horizon = policy.config.chunk_size
batch_size = 1

# Creating dummy actions (all zeros) for demonstration
# In reality, this would be your recorded robot data
dummy_actions = torch.zeros((batch_size, horizon, action_dim), device=device)
train_batch[ACTION] = dummy_actions

# Optional: Add mask for text loss (if you want to train the VLM part too)
# 1 = Calculate loss for this token, 0 = Ignore
# Typically you mask the prompt and only train on the generated response
train_batch["token_loss_mask"] = torch.ones_like(masks, device=device)

# 3. Run a Training Step
policy.train() # Switch to training mode
optimizer.zero_grad()

# The forward pass automatically calculates the loss
loss, loss_dict = policy(train_batch)

# Backward pass and optimization
loss.backward()
optimizer.step()

print("TRAINING STEP COMPLETE")
print(f"Total Loss: {loss.item():.4f}")
print(f"Detailed Losses: {loss_dict}")

In [ ]:
print("\n" + "="*80)
print("Testing sample_embedding...")
print("="*80)

with torch.no_grad():
    prefix_embeddings, past_key_values, prefix_pad_masks = policy.model.sample_embedding(
        processed_images,
        img_masks,
        tokens,
        masks
    )

    print(f"Prefix embeddings shape: {prefix_embeddings.shape}")
    print(f"Prefix pad masks shape: {prefix_pad_masks.shape}")
    print(f"Past key values type: {type(past_key_values)}")
    if past_key_values is not None:
        print(f"Past key values length: {len(past_key_values)}")
